#### Imports

In [ ]:
import pandas as pd
import numpy as np

from google.colab import drive

##Prophet
import matplotlib.pyplot as plt
from prophet import Prophet

##LSTM
import seaborn as sns
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Dense, Conv1D, LSTM, Input, TimeDistributed


In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')
data=pd.read_csv("~/Desktop/2023_1st_vacation/KRX_py/KRX_DATA/open/train.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#### slicing

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def getAOC(data):
  AOC = []
  close = data['종가']
  close_pre = close.shift(1)

  AOC.append(0)

  # aoc = (금일 종가 - 전일 종가) / 전일 종가
  for i in range(1, len(close)):
    aoc_value = ((close[i] - close_pre[i]) / close_pre[i]) * 100
    AOC.append(aoc_value)

  return AOC

#sort data by "종목명"
subject_Data=data.sort_values(by="종목명",inplace=False)
subject_Data.head()
subject_Data.columns

pv_s_data=pd.pivot_table(subject_Data,values=['거래량', '시가', '고가', '저가', '종가'],index=['종목코드','일자'])

divided = pv_s_data.index.get_level_values('종목코드').unique()

sliced_dataframes = []

for i in divided:
    sliced_data = pv_s_data.loc[i]
    sliced_dataframe = sliced_data.reset_index()
    sliced_dataframes.append(sliced_dataframe)

for i in range(len(sliced_dataframes)):
  sliced_dataframes[i]["AOC"] = getAOC(sliced_dataframes[i])

sliced_dataframes

[           일자     거래량     고가     시가     저가     종가       AOC
 0    20210601  114966  14700  14700  14450  14600  0.000000
 1    20210602  109559  14700  14700  14450  14500 -0.684932
 2    20210603   96158  14650  14550  14450  14600  0.689655
 3    20210604  133900  14800  14600  14550  14700  0.684932
 4    20210607  511140  15550  14800  14750  15150  3.061224
 ..        ...     ...    ...    ...    ...    ...       ...
 489  20230523  641524   9900   9160   9160   9770  5.280172
 490  20230524  205243   9820   9770   9550   9740 -0.307062
 491  20230525  398326  10180   9660   9660  10040  3.080082
 492  20230526  196257  10150  10050   9850   9850 -1.892430
 493  20230530  201361  10040   9960   9640   9700 -1.522843
 
 [494 rows x 7 columns],
            일자      거래량    고가    시가    저가    종가       AOC
 0    20210601   480096  1190  1180  1155  1175  0.000000
 1    20210602   928675  1225  1190  1180  1210  2.978723
 2    20210603   508487  1230  1220  1190  1200 -0.826446
 3    202

## Model
> prophet: need pip prophet


- predict end price for 15 days

#### [07/07]   
> Need to develop train/test split   
> [feat] learning complete

In [ ]:
class Stock_Prophet:
  def __init__(self,data) -> None:
    self.data=data
    self.forecasts=[]
    self.sliced=[]

  def train_test_split(self,ts_data: pd.DataFrame, n_test_rows: int) -> pd.DataFrame:
      train = ts_data.head(ts_data.shape[0] - n_test_rows)
      test = ts_data.tail(n_test_rows)
      return train.reset_index(), test.reset_index()

  def modeling(self):
    for i in self.sliced:
      prophet_model=Prophet()
      prophet_model.add_country_holidays(country_name="KR")
      prophet_model.fit(i)
      self.pred=prophet_model.make_future_dataframe(periods=30)
      forecast=prophet_model.predict(self.pred)
      '''!!! OLNY median value !!!'''
      forecast=forecast[["ds", "yhat"]]
      self.forecasts.append(forecast)

  def slice_n_prescale_data(self):

    def convert_date(date_str):
      return pd.to_datetime(date_str, format="%Y%m%d").strftime("%Y-%m-%d")

    ##make sliced=[sliced]
    self.data["일자"] = self.data["일자"].apply(convert_date)

    for idx in self.data.columns:
      slice_trading=self.data[["일자",idx]]
      slice_trading.columns=["ds","y"]
      self.sliced.append(slice_trading)

  def merge_and_return(self):
    for i in range(0,len(self.forecasts)):
      self.forecasts[i].columns=["일자",self.data.columns[i+1]]

    self.merged=pd.merge(self.forecasts[0],self.forecasts[1])

    for i in range(2,len(self.forecasts)):
      self.merged=pd.merge(self.merged,self.forecasts[i])

  def return_func(self):
    self.slice_n_prescale_data()
    self.modeling()
    self.merge_and_return()

    self.merged[["일자","종가"]]

    #plt.figure(figsize=(12, 9))
    #plt.plot(rescaled_real, label='actual')
    #plt.plot(rescaled_pred, label='prediction')
    #plt.legend()
    #plt.show()

    return self.merged[["일자","종가"]]





### Model learning TEST

In [ ]:
test=sliced_dataframes[0]

In [ ]:
prophet_trained=[]

sliced_model=Stock_Prophet(test)


In [ ]:
sliced_model.return_func()

ValueError: ignored

LSTM model
=========

In [ ]:
data=sliced_dataframes
data

In [ ]:
class LSTM_Molel:
  def __init__(self,data):
    self.data=data
    pass

  def prescale(self):
    pass

  def train_test_split(self):
    min_max_scaler=MinMaxScaler()


  def modeling(self):
    model=Sequential()
    model.add(LSTM,Dense(input_dim=6,activation='relu'))
    model.add(Dense=30,)




    pass

  def plot_res(self):
    pass